In [1]:
pip install langdetect

     |████████████████████████████████| 983kB 4.1MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=6aad98c4d9ddee59072b6af6e95cefd0a5ad88cc30f802bbd2bb11864835b335
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [2]:
# Import dependencies
from sklearn.preprocessing import LabelEncoder
from langdetect import detect, detect_langs
from google.colab import files
import pandas as pd
from itertools import islice
import time

In [3]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Shareable link
link = 'https://drive.google.com/file/d/1e2HjzljCHSDLqJWOMZ_noMzklSAgYTws/view?usp=sharing'
id = '1e2HjzljCHSDLqJWOMZ_noMzklSAgYTws'

In [5]:
# Read in DataFrame
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('song_lyrics_popularity.csv')  
song_lyrics_df = pd.read_csv('song_lyrics_popularity.csv')
song_lyrics_df.head(3)

,song,song_id,artist,artist_id,category,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.00000,0.366,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.00000,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1


In [6]:
# Remove the songs without lyrics
song_lyrics_df = song_lyrics_df.dropna()

In [7]:
# Add encoded category column
le = LabelEncoder()
new_column = le.fit_transform(song_lyrics_df['category']) 
song_lyrics_df.insert(5, 'category_id', new_column)

# Save DataFrame to CSV
song_lyrics_df.to_csv('encoded_df.csv', index=False)
song_lyrics_df.head(3)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.00000,0.366,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.00000,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1


In [8]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [43.6 kB]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Packa

In [9]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lyrics_NLP').getOrCreate()

In [10]:
# Read in DataFrame
music_df = spark.read.csv('encoded_df.csv', sep=',', header=True)
music_df.show(5)

+--------------------+--------------------+-------------+--------------------+--------+-----------+----------+--------------------+---------------------+---------------+------------+-----------------+-------------+--------------------+---------------------+-------------------------+-----------------+-------------------+--------------+--------------------+-----------------------+--------------------+---------------+
|                song|             song_id|       artist|           artist_id|category|category_id|popularity|              genres|audio_ft_danceability|audio_ft_energy|audio_ft_key|audio_ft_loudness|audio_ft_mode|audio_ft_speechiness|audio_ft_acousticness|audio_ft_instrumentalness|audio_ft_liveness|   audio_ft_valence|audio_ft_tempo|audio_ft_duration_ms|audio_ft_time_signature|              lyrics|non_alpha_words|
+--------------------+--------------------+-------------+--------------------+--------+-----------+----------+--------------------+---------------------+---------

In [11]:
# Import functions
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

In [12]:
# Create all the features to the data set
tokenizer = Tokenizer(inputCol='lyrics', outputCol='words')
stop_remover = StopWordsRemover(inputCol='words', outputCol='filtered')

In [13]:
# Create and run a data processing Pipeline
data_prep_pipeline = Pipeline(stages=[tokenizer, stop_remover])

In [14]:
# Fit and transform the pipeline
remover = data_prep_pipeline.fit(music_df)
removed_df = remover.transform(music_df)
removed_df.show(5)

+--------------------+--------------------+-------------+--------------------+--------+-----------+----------+--------------------+---------------------+---------------+------------+-----------------+-------------+--------------------+---------------------+-------------------------+-----------------+-------------------+--------------+--------------------+-----------------------+--------------------+---------------+--------------------+--------------------+
|                song|             song_id|       artist|           artist_id|category|category_id|popularity|              genres|audio_ft_danceability|audio_ft_energy|audio_ft_key|audio_ft_loudness|audio_ft_mode|audio_ft_speechiness|audio_ft_acousticness|audio_ft_instrumentalness|audio_ft_liveness|   audio_ft_valence|audio_ft_tempo|audio_ft_duration_ms|audio_ft_time_signature|              lyrics|non_alpha_words|               words|            filtered|
+--------------------+--------------------+-------------+--------------------+

In [15]:
# Convert DataFrame to Pandas
nlp_df = removed_df.toPandas()
nlp_df.head(3)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"[, im, like, the, water, when, your, ship, rol...","[, im, like, water, ship, rolled, night, rough..."
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.284,0.0805,0.0,0.366,0.7559999999999999,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"[, didnt, even, wanna, go, out, whyd, you, cal...","[, didnt, even, wanna, go, whyd, call, ?, ive,..."
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.213,0.0,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"[, you, cut, out, a, piece, of, me, and, now, ...","[, cut, piece, bleed, internally, left, withou..."


In [16]:
# Remove instrumental songs
count = 0
for index, row in nlp_df.iterrows():
    if row['lyrics'].lower() == 'instrumental':
        count+=1
        nlp_df = nlp_df.drop(index)
print(f'Removed {count} instrumental songs.')
nlp_df.head(3)

Removed 167 instrumental songs.


,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"[, im, like, the, water, when, your, ship, rol...","[, im, like, water, ship, rolled, night, rough..."
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.284,0.0805,0.0,0.366,0.7559999999999999,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"[, didnt, even, wanna, go, out, whyd, you, cal...","[, didnt, even, wanna, go, whyd, call, ?, ive,..."
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.213,0.0,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"[, you, cut, out, a, piece, of, me, and, now, ...","[, cut, piece, bleed, internally, left, withou..."


In [ ]:
# Create a list of languages
t0 = time.time()
languages = []
for index, row in nlp_df.iterrows():
    lyrics = row['lyrics']
    try:
        langs = detect_langs(lyrics)
        languages.append((len(langs), str(langs[0])[0:2]))
    except:
        languages.append(None)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(len(languages))
unique_lang_count = len(set([lang[1] for lang in languages if lang]))
print(f'{unique_lang_count} unique languages found.')

Run time: 63.18269181251526 seconds
8444
23 unique languages found.


In [ ]:
# Add a language column to the DataFrame
nlp_df['language'] = languages

In [ ]:
# Find songs indeces with one language that is not English
no_lang = []
indeces = []
langs_not_english = []
for index, row in nlp_df.iterrows():
  lang = row['language']
  if lang == None:
    no_lang.append(index)
  if lang != None:
    if (lang[0] == 1) & (lang[1] != 'en'):
      indeces.append(index)
    if (lang[0] > 1) & (lang[1] != 'en'):
      langs_not_english.append(index)
print(len(no_lang))
print(len(indeces))
len(langs_not_english)

9
85


44

In [ ]:
# Remove songs with no language and songs with one language that is not English
nlp_df = nlp_df.drop(no_lang)
nlp_df = nlp_df.drop(indeces)
print(len(nlp_df))
nlp_df.head(3)

8350


,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered,language
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"[, im, like, the, water, when, your, ship, rol...","[, im, like, water, ship, rolled, night, rough...","(1, en)"
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.284,0.0805,0.0,0.366,0.7559999999999999,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"[, didnt, even, wanna, go, out, whyd, you, cal...","[, didnt, even, wanna, go, whyd, call, ?, ive,...","(1, en)"
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.213,0.0,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"[, you, cut, out, a, piece, of, me, and, now, ...","[, cut, piece, bleed, internally, left, withou...","(1, en)"


In [ ]:
# Find the number of songs that do not have English lyrics
len(nlp_df[nlp_df['language'].apply(lambda x: 'en' not in x)])

44

In [ ]:
# Save DataFrame to CSV
nlp_df.drop('words', axis=1).to_csv('nlp_df.csv', index=False)
files.download('nlp_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Create a list of co-occurring words
cooccurring_words = []
for index, row in nlp_df.iterrows():
    lyrics = row['filtered']
    while '' in lyrics:
      lyrics.remove('')
    while ' ' in lyrics:
      lyrics.remove(' ')
    for i in range(len(lyrics)-1):
      cooccurring_words.append(lyrics[i] + ' ' + lyrics[i+1])
print(len(cooccurring_words))
len(set(cooccurring_words))

1376873


485774

In [ ]:
# Create a list of uncommon words from the re-occurring words list
t0 = time.time()
unique_cooccurring_words = list(set(cooccurring_words))
count = 0
uncommon_words = []
for words in unique_cooccurring_words[0:10000]:
  freq = cooccurring_words.count(words)
  if freq < 100:
    uncommon_words.append(words)
  else: 
    count+=1
    print(words, freq)
t1 = time.time()
print(f'Run time to create list of uncommon words: {t1-t0} seconds')
print(f'{count} of the first 10,000 unique co-occurring words appeared at least 100 times.')

# Create a new list without the uncommon words
t0 = time.time()
new_list = [words for words in cooccurring_words if words not in uncommon_words]
t1 = time.time()
print(f'Run time to create new list: {t1-t0} seconds')
len(new_list)

Run time to create list of uncommon words: -0.0002620220184326172 seconds
13 of the first 10,000 unique co-occurring words appeared at least 100 times.
Run time to create new list: 376.57666206359863 seconds


1351629

In [ ]:
# Create DataFrame of co-occurring words
t0 = time.time()
unique_cooccurring_words = list(set(new_list))
print(len(unique_cooccurring_words))
cooccurring_words_df = pd.DataFrame(columns=unique_cooccurring_words)
cooccurring_words_df.loc[0] = 0
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
cooccurring_words_df

475787
Run time: 25.537651300430298 seconds


,oh cant,story enough,im gone,strait wheres,dont ever,right ?,love way,needle dropping,something runs,fresh baby,ill go,im like,id pound,? really,way way,? love,like ooh,dont like,shoulder narrow,trial white,right right,mind save,burdensome oh,freezing scantily,matter versus,court reported,walk valley,babies teach,mom ?,chase wanna,dumped husband,youre smartly,erotic zone,might believe,caught flames,within words,nigga listen,girl doesnt,taxes staring,halftime im,...,still things,sheets know,rapper ?,paphos city,men ruthless,bitch life,got drama,burn tongue,brace dear,meat like,woo tripping,vultures depths,holding middle,rip eyes,? exactly,dream stayed,thats sound,wiggle right,takin back,mind murda,thick lips,636 20,father stepfather,content void,hunger strike,say guess,come memories,quite bizzare,super cool,thirteen month,f baby,swift sharp,memory used,department monitored,space isnt,breeze beyotch,trying hardest,dog streets,stuck surface,boy year
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Fill DataFrame with co-occurring word frequencies
t0 = time.time()
t2 = time.time()
count = 0
for index, words in enumerate(new_list):
  cooccurring_words_df.loc[0, words]+=1
  if (index+1) % 100000 == 0:
    print(f'Run time for co-occurring words {count*100000}-{(count+1)*100000}: {time.time()-t2}')
    count+=1
    t2 = time.time()
t1 = time.time()
print(f'Total run time: {t1-t0} seconds')
cooccurring_words_df

Run time for co-occurring words 0-100000: 373.8604369163513
Run time for co-occurring words 100000-200000: 373.28043389320374
Run time for co-occurring words 200000-300000: 408.2024862766266
Run time for co-occurring words 300000-400000: 443.5948028564453
Run time for co-occurring words 400000-500000: 436.78964400291443
Run time for co-occurring words 500000-600000: 468.58050537109375
Run time for co-occurring words 600000-700000: 333.27354192733765
Run time for co-occurring words 700000-800000: 343.82261538505554
Run time for co-occurring words 800000-900000: 414.14837312698364
Run time for co-occurring words 900000-1000000: 432.89326095581055
Run time for co-occurring words 1000000-1100000: 357.5977351665497
Run time for co-occurring words 1100000-1200000: 297.38461661338806
Run time for co-occurring words 1200000-1300000: 294.08880066871643
Total run time: 5134.9790234565735 seconds


,oh cant,story enough,im gone,strait wheres,dont ever,right ?,love way,needle dropping,something runs,fresh baby,ill go,im like,id pound,? really,way way,? love,like ooh,dont like,shoulder narrow,trial white,right right,mind save,burdensome oh,freezing scantily,matter versus,court reported,walk valley,babies teach,mom ?,chase wanna,dumped husband,youre smartly,erotic zone,might believe,caught flames,within words,nigga listen,girl doesnt,taxes staring,halftime im,...,still things,sheets know,rapper ?,paphos city,men ruthless,bitch life,got drama,burn tongue,brace dear,meat like,woo tripping,vultures depths,holding middle,rip eyes,? exactly,dream stayed,thats sound,wiggle right,takin back,mind murda,thick lips,636 20,father stepfather,content void,hunger strike,say guess,come memories,quite bizzare,super cool,thirteen month,f baby,swift sharp,memory used,department monitored,space isnt,breeze beyotch,trying hardest,dog streets,stuck surface,boy year
0,147,1,157,1,181,153,197,1,1,1,147,196,1,102,169,157,103,205,1,1,247,2,1,1,1,24,4,2,4,1,7,1,1,2,4,1,1,1,1,1,...,1,3,1,1,1,1,5,1,4,1,1,1,3,3,1,1,6,12,1,1,1,1,1,1,1,4,1,4,1,2,1,1,1,1,1,1,1,1,1,1


In [ ]:
# Check that the frequencies add up correctly
total = sum(cooccurring_words_df.loc[0])
print(f'Sum: {total}')
# should equal len(new_list)

Sum: 1351629


In [ ]:
# Remove columns of words that only appear once
cooccurring_words_df = cooccurring_words_df.drop(columns=cooccurring_words_df.columns[(cooccurring_words_df[0:1] == 1).any()])    
cooccurring_words_df

,oh cant,im gone,dont ever,right ?,love way,ill go,im like,? really,way way,? love,like ooh,dont like,right right,mind save,court reported,walk valley,babies teach,mom ?,dumped husband,might believe,caught flames,fall feel,lovely daughter,wind turn,really yeah,come rain,devil wears,especially get,happen fear,guy course,drop kentucky,everybody gonna,hostile land,fuck bitches,see times,calm storm,sit take,shackled gto,internalize everything,long tell,...,dark endless,crawling back,wholl walk,talkin walkin,obey !,command members,apathy feeding,cold genuine,mir die,wish stay,shelter whenever,youre skin,time hell,10 comin,didnt cap,im asleep,period thought,death beyond,black jack,anguish funny,getting flips,heard heard,able buzz,headed west,little punk,affiant convinced,nobodys gotta,new album,ride cause,fly broken,sheets know,got drama,brace dear,holding middle,rip eyes,thats sound,wiggle right,say guess,quite bizzare,thirteen month
0,147,157,181,153,197,147,196,102,169,157,103,205,247,2,24,4,2,4,7,2,4,6,2,2,6,16,5,6,5,8,3,5,3,42,2,4,3,2,2,10,...,2,6,3,2,3,2,4,3,3,10,2,6,12,6,2,5,4,4,5,2,6,8,2,3,5,2,3,2,9,5,3,5,4,3,3,6,12,4,4,2


In [ ]:
# Create filtered DataFrame with co-occuring words that appear at least 250 times
filtered_cooccurring_words_df = cooccurring_words_df.drop(columns=cooccurring_words_df.columns[(cooccurring_words_df[0:1]<250).any()])    
filtered_cooccurring_words_df = filtered_cooccurring_words_df.sort_values(by=[0], axis=1, ascending=False)
filtered_cooccurring_words_df

,oh oh,yeah yeah,ooh ooh,na na,dont know,la la,dont wanna,let go,im gonna,oh yeah,ah ah,know im,love love,doo doo,dont want,hey hey,ive got,feel like,da da,cause im,know know,dont need,get get,every time,like im,dont stop,cant get,yeah im,dont care,know youre,cant see,think im,go go,oh baby,? im,baby baby,? dont,dont let,im still,know dont,...,close eyes,youll never,wont let,cant stop,? ?,wanna see,im coming,night long,? cause,cause know,thats way,love im,gotta get,dont get,get back,love baby,know got,im ready,ha ha,one thing,yeah know,? know,run away,cant help,hey !,ya ya,know love,nah nah,woah oh,cant take,good time,dont mind,cause youre,last night,dont worry,brand new,like dont,know like,im feeling,feel good
0,4770,3231,2572,2425,2384,2343,1519,1238,1208,1075,1032,1006,994,983,982,963,945,927,925,873,761,724,643,615,596,588,577,547,530,520,507,502,500,498,483,481,477,464,459,457,...,300,300,299,296,294,294,294,293,291,288,288,286,283,283,283,283,282,281,276,276,275,271,271,269,268,268,267,264,263,262,260,258,257,257,256,255,254,253,251,250


In [ ]:
# Save DataFrame to CSV
filtered_cooccurring_words_df.to_csv('top_cooccurring_TF.csv', index=False)
files.download('top_cooccurring_TF.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>